In [16]:
import dash
from dash import dcc, html
from dash.dependencies import Input, Output
import pandas as pd
import plotly.express as px
import dash_bootstrap_components as dbc
import dizionari
import datetime #TODO: eliminarla se non utilizzata
import numpy as np


data = pd.read_csv("../data/ISTAT_Italian_Excursion_2014_2022.csv")

# CREO UN NUOVO DATAFRAME PIU' VERSATILE
# Nota: Nel nostro dataframe non sono presenti valori nulli [data.isnull().sum()]

# Rimozione di colonne ridondanti e superflui
colonne_da_eliminare = ["ID", "progind", "sesso", "staciv4", "condogg", "posiz4", "ateco3", "cond4", "ORARIO", "RAPP", "CODPROF7", "progesc", "PARTESC", "NPARTESC"]
df = data.drop(columns=colonne_da_eliminare)

# Rinomina delle colonne
df.rename(columns = dizionari.dizionario_nomi, inplace=True)

# Creazione di una colonna 'Data' nel formato (ANNO-mese)
df['Data'] = pd.to_datetime(df['Anno'].apply(str) + '-' + df['Mese'].apply(str), format='%Y-%m')
df["Data"] = df["Data"].dt.strftime("%Y-%m")

# Colonna Età nel formato (anno)
df['Età'] = df['Età'].map(dizionari.eta_mapping)
df['Regione'] = (df['Regione']/10).astype(int)
df

,Mese,Anno,Età,Regione,Zona d'Italia,Livello di istruzione,Italia/Estero,Destinazione (regioni italiane o paesi esteri),Destinazione (città),Motivo dell escursione,Mezzo,Stagione,Data
0,1,2014,<=14,17,4,1,1,17,77,5,5,1,2014-01
1,3,2014,<=14,18,4,1,1,18,78,3,5,1,2014-03
2,7,2014,35-44,15,4,2,1,15,63,1,3,3,2014-07
3,7,2014,35-44,15,4,2,1,15,63,7,5,3,2014-07
4,3,2014,>=75,4,2,1,1,4,22,2,5,1,2014-03
...,...,...,...,...,...,...,...,...,...,...,...,...,...
37824,1,2022,45-54,3,1,2,1,2,7,1,5,1,2022-01
37825,8,2022,35-44,12,3,4,1,12,59,1,5,3,2022-08
37826,12,2022,55-64,4,2,3,1,4,21,9,5,4,2022-12
37827,12,2022,55-64,4,2,3,1,4,21,9,5,4,2022-12


In [19]:
app = dash.Dash(__name__, external_stylesheets=[dbc.themes.MATERIA])

radio_linechart = dbc.RadioItems(
    id="radio-linecharts-id",
    options=[
        {"label": "Età", "value": "Età"},
        {"label": "Estero", "value": "Destinazione (regioni italiane o paesi esteri)"},
    ],
    value="Età",
    inline=True
)

map_feature_dropdown = dcc.Dropdown(
    id='map-id',
    options=[
        {'label': 'Partenze', 'value': 'Partenze'},
        {'label': 'Arrivi', 'value': 'Arrivi'}],
    value='Partenze'  # Valore predefinito
)

violin_feature_dropdown = dcc.Dropdown(
    id='violin-feature',
    options=[{'label': col, 'value': col} for col in df.columns],
    value='Mese'  # Valore predefinito
)

variabili_checklist = dbc.Checklist(
    id='correlation-checklist',
    options=[{'label': col, 'value': col} for col in df.columns],
    value=['Mese', 'Anno', 'Data'],  # Valori predefiniti
    inline=True
)
# Layout dell'app
app.layout = dbc.Container([
    html.H1("Analisi escursioni Italia"),
    dbc.Row([
        dbc.Col(
            dbc.Stack([
              html.Label("Seleziona la caratteristica dell'asse Y:"),
              radio_linechart,
              ])
            , width=6),
        dbc.Col(
            dbc.Stack([
              html.Label("Seleziona la caratteristica dell'asse Y:"),
              map_feature_dropdown,
            ])
        , width=6, align="center"),
    ]),
    dbc.Row([
        dbc.Col(
            dbc.Stack([
              dcc.Graph(id='graph')
              ])
            , width=6),
        dbc.Col(
            dbc.Stack([
              dcc.Graph(id='map')
            ])
        , width=6),
    ]),
    dbc.Row([
        dbc.Col([
            dbc.Stack([
                html.Label("Select a feature to visualize with a violin plot:"),
                violin_feature_dropdown,
            ])
        ], width=4, align="center"),
        dbc.Col([
            dbc.Stack([
              html.Label("Seleziona le variabili per la heatmap di correlazione:"),
              variabili_checklist,
            ])
        ], width=8, align="center"),
    ]),
    dbc.Row([
        dbc.Col([
            dbc.Stack([
                dcc.Graph(id='violin-plot')
            ])
        ], width=4),
        dbc.Col([
            dbc.Stack([
              dcc.Graph(id='heatmap')
            ])
        ], width=8),
    ]),
])

# LINECHART - Evoluzione degli spostamenti negli anni (suddivisi per: [Età, Stati Esteri])
# Callback per il grafico a dispersione (scatter plot)
@app.callback(
    Output('graph', 'figure'),
    [Input('radio-linecharts-id', 'value')]
)
def update_line_chart(radio_linechart):
    conteggio = df.groupby(['Anno', radio_linechart]).size().reset_index(name='Numero escursioni')
    colori = ['#FF0000', '#FF4500', '#FF8C00', '#FFD700', '#ADFF2F', '#7FFF00', '#32CD32', '#008000']

    if radio_linechart == "Destinazione (regioni italiane o paesi esteri)":
        radio_linechart = "Stati Esteri"
        conteggio.rename(columns={"Destinazione (regioni italiane o paesi esteri)" : radio_linechart}, inplace=True)
        conteggio = conteggio[conteggio[radio_linechart] > 100]
        conteggio['Stati Esteri'] = conteggio[radio_linechart].map(dizionari.destinazione_regioni_stato_mapping)
        
        colori = [
            '#FF5733', '#33FF57', '#3357FF', '#FF33A6', '#A633FF', '#33FFA6',
            '#FF8C33', '#8CFF33', '#338CFF', '#FF338C', '#8C33FF', '#33FF8C',
            '#FF5733', '#33FF57', '#3357FF', '#FF33A6', '#A633FF', '#33FFA6',
            '#FF8C33', '#8CFF33', '#338CFF', '#FF338C', '#8C33FF', '#33FF8C',
            '#FF5733', '#33FF57', '#3357FF', '#FF33A6'
            ]

    fig = px.line(conteggio,
                  x='Anno',
                  y='Numero escursioni',
                  color = radio_linechart,
                  title = f'Evoluzione degli spostamenti negli anni rispetto a [{radio_linechart}]',
                  color_discrete_sequence=colori)

    fig.update_layout(margin=dict(l=50, r=50, t=50, b=50))
    
    return fig


@app.callback(
    Output('map', 'figure'),
    [Input('map-id', 'value')]
)
def update_map(map_feature_dropdown):
    if map_feature_dropdown == "Partenze":
        conteggio_partenze = df['Regione'].value_counts().reset_index(name="Numero escursioni")
        print(conteggio_partenze)
        fig = px.choropleth(
            conteggio_partenze,
            geojson='https://raw.githubusercontent.com/openpolis/geojson-italy/master/geojson/limits_IT_regions.geojson',
            locations='Regione',
            color='Numero escursioni',
            color_continuous_scale='Blues',
            featureidkey='properties.reg_istat_code_num',
            #animation_frame='Date',
            range_color=(0, max(conteggio_partenze['Numero escursioni']))
            )
        
    elif map_feature_dropdown == "Arrivi":
        conteggio_arrivi = df['Destinazione (città)'].value_counts().reset_index(name="Numero escursioni")
        
        print(conteggio_arrivi)
        
        fig = px.choropleth(
            conteggio_arrivi,
            geojson='https://raw.githubusercontent.com/openpolis/geojson-italy/master/geojson/limits_IT_provinces.geojson',
            locations='Destinazione (città)',
            color='Numero escursioni',
            color_continuous_scale='Reds',
            featureidkey='properties.prov_istat_code_num',
            #animation_frame='Date',
            range_color=(0, max(conteggio_arrivi['Numero escursioni']))
            )
        
    fig.update_layout(margin={"r": 0, "t": 0, "l": 0, "b": 0})
    fig.update_geos(fitbounds="locations", visible=True)
    
    return fig


# Funzione per la generazione dei colori della mappa (presa da internet)
def generateColorScale(colors, naColor):
    colorArray=[]
    colorArray.append([0,naColor])
    for grenze, color in zip(np.linspace(0.01,1,len(colors)), colors):
        colorArray.append([grenze, color])
    return colorArray


# Esecuzione dell'app
if __name__ == '__main__':
    app.run_server(debug=True, jupyter_mode="external")





Dash app running on http://127.0.0.1:8050/


    Regione  Numero escursioni
0         5               4949
1         4               3577
2         3               3495
3         8               3432
4         6               3207
5         9               3011
6        15               2601
7         7               1902
8        12               1878
9         1               1612
10       11               1387
11       10               1316
12        2                915
13       19                863
14       20                713
15       18                668
16       17                644
17       16                607
18       13                564
19       14                488
     Destinazione (città)  Numero escursioni
0                      27               1951
1                      58               1573
2                      22               1564
3                      21               1436
4                      23               1425
..                    ...                ...
108                    85         